<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.64
    Uninstalling yfinance-0.2.64:
      Successfully uninstalled yfinance-0.2.64
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-04 13:20:07
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.36 C
-------------------
Today PnL: -11.79 K (-0.09%)
Current PnL: -13.50 L (-9.4%)
CY Booked + Current PnL: -6.76 L (-4.71%)
-------------------
Total profit:  4.49 L
Total loss:  -17.98 L
-------------------
Total Booked + Current PnL: 20.22 L (17.35%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (4.94%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 71.91 L (52.72%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.68%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,HINDALCO,651.95,761.55,-19.08,H-LC,84.38,111112.0,6800.0,10733.0,0.23,6.52,9.66,16.81,27.0,0.63,0.81,23.61,X5K,ATH,METALS
5,APOLLOHOSP,6904.43,8285.00,-16.96,H-LC,85.42,113078.0,9512.0,11195.0,-0.35,9.18,9.90,20.00,30.0,0.85,0.83,24.55,X40N,BTT,HEALTHCARE
3,ADANIPORTS,1103.69,1583.00,0.58,M-LC,27.08,100863.0,22501.0,11529.0,-0.73,28.71,11.43,43.43,77.0,1.95,0.74,35.56,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,0.15,M-LC,30.21,173415.0,32575.0,11914.0,0.61,23.13,6.87,31.59,89.0,2.73,1.27,50.64,XY24,NTT,BANKS
87,UJJIVANSFB,52.77,53.00,49.36,M-SC,87.50,129168.0,-13311.0,13937.0,-2.61,-9.34,10.79,0.44,243.0,-0.96,0.95,52.45,OX40N,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-29.18,L-SC,100.00,142297.0,455.0,56122.0,0.64,0.32,39.44,39.88,259.0,0.01,1.04,44.21,XY24,NTT,POWER
12,BANKINDIA,116.91,190.00,-31.29,M-MC,18.75,180669.0,861.0,111545.0,-0.86,0.48,61.74,62.52,171.0,0.01,1.33,34.73,XR,NTT,BANKS
55,LALPATHLAB,2804.19,3545.00,-33.01,H-SC,25.00,154214.0,-16.0,40759.0,1.71,-0.01,26.43,26.42,125.0,-0.00,1.13,21.61,X40N,NTT,HEALTHCARE
38,HEROMOTOCO,4311.81,6039.03,-5.24,H-MC,58.33,152355.0,1442.0,59007.0,0.98,0.96,38.73,40.06,102.0,0.02,1.12,24.62,AR,ATH,AUTO


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,10.42,48916.0,7661.0,3145.0,4.60,18.57,6.43,26.19,1.0,2.44,0.36,15.16,XY24,ATH,IT
65,RELAXO,902.64,1176.00,-35.51,H-SC,61.46,58310.0,-57228.0,92217.0,4.10,-49.53,158.15,30.28,134.0,-0.62,0.43,13.92,X40N,NTT,FOOTWEAR
33,GREENPANEL,375.16,537.00,215.25,M-SC,86.46,140666.0,-42412.0,121395.0,3.67,-23.17,86.30,43.14,216.0,-0.35,1.03,32.32,XY24,NTT,MISC
70,SATIN,199.77,274.00,-14.59,H-SC,88.54,167986.0,-27988.0,100808.0,2.87,-14.28,60.01,37.16,130.0,-0.28,1.23,25.45,XY24,NTT,FINANCE
90,VALIANTORG,512.64,838.00,-360.76,H-SC,8.33,110760.0,-22526.0,107116.0,2.35,-16.90,96.71,63.47,148.0,-0.21,0.81,85.66,XR,NTT,CHEMICALS


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SAMMAANCAP,170.35,326.00,-164.77,M-SC,35.42,74340.0,-27870.0,121263.0,-8.72,-27.27,163.12,91.37,206.0,-0.23,0.55,20.68,XY25,NTT,FINANCE
11,BANDHANBNK,214.25,400.00,-30.37,H-SC,69.79,206654.0,-41876.0,257346.0,-3.14,-16.85,124.53,86.70,129.0,-0.16,1.52,36.71,XY24,NTT,BANKS
87,UJJIVANSFB,52.77,53.00,49.36,M-SC,87.50,129168.0,-13311.0,13937.0,-2.61,-9.34,10.79,0.44,243.0,-0.96,0.95,52.45,OX40N,NTT,BANKS
30,DMART,3764.92,5201.00,-16.79,X-LC,62.50,208980.0,24499.0,45871.0,-1.93,13.28,21.95,38.14,15.0,0.53,1.53,25.29,X40N,NTT,FMCG
4,ALKYLAMINE,2347.98,4546.37,6.25,X-SC,98.96,98866.0,-2097.0,96632.0,-1.79,-2.08,97.74,93.63,64.0,-0.02,0.73,47.26,SR,ATH,CHEMICALS


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,49.36,M-SC,87.50,129168.0,-13311.0,13937.0,-2.61,-9.34,10.79,0.44,243.0,-0.96,0.95,52.45,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-30.70,M-SC,11.46,147768.0,-23373.0,23318.0,-0.11,-13.66,15.78,-0.03,189.0,-1.00,1.08,6.20,OX40N,NTT,DURABLES
92,VIPIND,488.80,489.0,-212.65,H-SC,96.88,81296.0,-14020.0,14056.0,0.93,-14.71,17.29,0.04,154.0,-1.00,0.60,60.35,OX40N,NTT,MISC
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,91.67,148800.0,-25759.0,25787.0,-0.11,-14.76,17.33,0.02,127.0,-1.00,1.09,29.03,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-17.31,X-MC,16.67,86800.0,-15852.0,15850.0,-0.24,-15.44,18.26,-0.00,58.0,-1.00,0.64,31.95,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,10.42,48916.0,7661.0,3145.0,4.60,18.57,6.43,26.19,1.0,2.44,0.36,15.16,XY24,ATH,IT
83,TCS,3794.03,4998.00,-18.06,X-LC,42.71,266830.0,-29104.0,123009.0,0.59,-9.83,46.10,31.73,1.0,-0.24,1.96,6.80,X40,BTT,IT
48,INFY,1461.46,2275.00,-9.55,X-LC,67.71,289747.0,28146.0,117463.0,0.51,10.76,40.54,55.67,2.0,0.24,2.13,17.52,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-16.71,X-LC,29.17,260803.0,-9544.0,44180.0,0.66,-3.53,16.94,12.81,6.0,-0.22,1.91,9.07,XY25,NTT,FMCG
61,NESTLEIND,2268.60,2755.00,-12.22,X-LC,31.25,278343.0,12917.0,43978.0,0.03,4.87,15.80,21.44,9.0,0.29,2.04,11.83,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,UNIONBANK,123.87,163.0,0.15,M-LC,30.21,173415.0,32575.0,11914.0,0.61,23.13,6.87,31.59,89.0,2.73,1.27,50.64,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,0.58,M-LC,27.08,100863.0,22501.0,11529.0,-0.73,28.71,11.43,43.43,77.0,1.95,0.74,35.56,XY24,NTT,MISC
32,ENRIN,1377.95,2975.9,NaN,NaN,72.92,73245.0,38796.0,1150.0,0.26,112.62,1.57,115.97,NaN,33.74,0.54,12.00,AR,ATH,ELECTRICAL


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,92.75,M-MC,48.96,230920.0,5958.0,161690.0,-1.76,2.65,70.02,74.53,185.0,0.04,1.69,34.07,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.64,27.50,H-SC,51.04,130423.0,5126.0,53956.0,0.84,4.09,41.37,47.15,139.0,0.10,0.96,63.17,MH,ATH,POWER
20,CAMS,3643.00,5250.99,4.54,H-SC,37.50,117348.0,15344.0,29677.0,-1.58,15.04,25.29,44.14,124.0,0.52,0.86,33.73,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,0.58,M-LC,27.08,100863.0,22501.0,11529.0,-0.73,28.71,11.43,43.43,77.0,1.95,0.74,35.56,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,0.15,M-LC,30.21,173415.0,32575.0,11914.0,0.61,23.13,6.87,31.59,89.0,2.73,1.27,50.64,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,72.92,73245.0,38796.0,1150.0,0.26,112.62,1.57,115.97,NaN,33.74,0.54,12.00,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-38.30,M-SC,89.58,203419.0,31798.0,130270.0,-1.28,18.53,64.04,94.43,198.0,0.24,1.49,50.14,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-2.41,M-LC,83.33,167121.0,16176.0,93287.0,0.94,10.72,55.82,72.51,69.0,0.17,1.23,15.12,XR,NTT,IT
54,KPIGREEN,497.21,731.64,27.50,H-SC,51.04,130423.0,5126.0,53956.0,0.84,4.09,41.37,47.15,139.0,0.10,0.96,63.17,MH,ATH,POWER
45,INDIAMART,2327.09,4850.92,-48.26,H-SC,80.21,134885.0,11549.0,122219.0,-1.36,9.36,90.61,108.45,122.0,0.09,0.99,33.81,AR,ATH,MISC


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.55,NaN,NaN,7.29,17605.0,-31988.0,910.0,2.04,-64.50,5.17,-62.66,NaN,-35.15,0.13,18.06,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,95.83,67565.0,-45984.0,86037.0,0.31,-40.50,127.34,35.27,261.0,-0.53,0.50,65.37,XR,NTT,HOTELS
7,ASIANTILES,75.41,137.00,7244.44,M-SC,93.75,80611.0,-13199.0,89817.0,1.52,-14.07,111.42,81.67,242.0,-0.15,0.59,56.11,XR,NTT,CERAMICS
58,MASFIN,326.60,399.50,-18.25,H-SC,78.12,92715.0,-5265.0,27138.0,0.54,-5.37,29.27,22.32,137.0,-0.19,0.68,34.47,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,6.25,X-SC,98.96,98866.0,-2097.0,96632.0,-1.79,-2.08,97.74,93.63,64.0,-0.02,0.73,47.26,SR,ATH,CHEMICALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-29.18,L-SC,100.00,142297.0,455.0,56122.0,0.64,0.32,39.44,39.88,259.0,0.01,1.04,44.21,XY24,NTT,POWER
4,ALKYLAMINE,2347.98,4546.37,6.25,X-SC,98.96,98866.0,-2097.0,96632.0,-1.79,-2.08,97.74,93.63,64.0,-0.02,0.73,47.26,SR,ATH,CHEMICALS
15,BLUESTARCO,1646.70,2337.55,-2.82,H-SC,97.92,182310.0,17640.0,51448.0,-1.04,10.71,28.22,41.95,121.0,0.34,1.34,18.98,X40N,ATH,AC
92,VIPIND,488.80,489.00,-212.65,H-SC,96.88,81296.0,-14020.0,14056.0,0.93,-14.71,17.29,0.04,154.0,-1.00,0.60,60.35,OX40N,NTT,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,95.83,67565.0,-45984.0,86037.0,0.31,-40.50,127.34,35.27,261.0,-0.53,0.50,65.37,XR,NTT,HOTELS


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.89
1,25,41.84
2,50,70.42


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.44
LC    33.92
MC    21.58
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.48
X40      12.28
XY25     11.95
X40N     11.91
XR       10.15
AR        8.21
OX40N     8.21
MH        1.70
X5K       1.46
X200      1.40
SR        1.22
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.64
M-SC    15.96
H-LC    14.12
X-LC    12.12
H-MC     9.53
X-MC     7.62
M-LC     6.64
X-SC     4.39
M-MC     4.07
L-SC     2.45
L-LC     1.04
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.05,-1.95,36.04
IT,12.13,-9.92,63.92
FINANCE,7.86,-12.50,53.14
BANKS,7.82,-7.13,50.84
MISC,6.64,-14.91,68.11
PAINTS,5.72,-11.82,33.86
ELECTRICAL,5.53,2.06,43.68
HEALTHCARE,4.56,-3.60,33.86
AUTO,4.34,-14.20,64.42


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2706325.0,27
XR,1075333.0,13
AR,1024308.0,10
X40,645936.0,9
X40N,539864.0,11
XY25,489994.0,8
OX40N,310354.0,11
SR,177912.0,2
MH,83547.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1968024.0,20
M-SC,1764368.0,21
H-LC,612790.0,12
H-MC,533734.0,8
X-LC,523333.0,7
M-MC,415001.0,3
X-MC,414148.0,6
X-SC,320783.0,5
M-LC,268176.0,5


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      875791.0      6
M-SC       XY24      808570.0      7
H-SC       AR        523286.0      3
M-SC       XR        333446.0      4
M-MC       XY24      303456.0      2
H-MC       XY24      299408.0      3
X-LC       X40       240472.0      2
H-SC       X40N      214101.0      4
           XR        211988.0      3
H-LC       X40N      195934.0      4
M-SC       AR        182839.0      2
           OX40N     165637.0      5
M-LC       XY24      160248.0      3
H-LC       X40       155597.0      3
L-SC       XR        147675.0      2
X-MC       X40       147401.0      2
X-SC       XR        123391.0      1
M-SC       XY25      121263.0      1
X-LC       XY25      119337.0      3
           AR        117653.0      1
M-MC       XR        111545.0      1
X-MC       XY25      103911.0      1
X-SC       SR         96632.0      1
X-MC       XY24       93960.0      1
M-LC       XR         93287.0      1
H-SC       OX40N      88902.0      3
H-MC       XY25       83663.0      1
M-SC       SR         81280.0      1
H-LC       AR         80373.0      1
M-SC       X40        71333.0      1
H-LC       X200       69136.0      1
           X5K        67830.0      2
X-SC       XY24       60795.0      1
L-SC       AR         60000.0      1
H-MC       AR         59007.0      1
L-SC       XY24       56122.0      1
L-MC       XR         54001.0      1
H-SC       MH         53956.0      1
X-MC       X40N       53026.0      1
L-LC       XY25       47179.0      1
X-LC       X40N       45871.0      1
H-LC       XY24       43920.0      1
X-SC       OX40N      39965.0      2
H-MC       X40        31133.0      1
           X40N       30932.0      1
           MH         29591.0      1
X-MC       OX40N      15850.0      1
M-LC       XY25       14641.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
